In [1]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth', 'Utils', 'Other', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = '10 Industry Portfolios - Average Value Weighted Returns.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [3]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    rf = pd.read_csv(file_name, delimiter = ',', header = 0, usecols = ['RF3'])
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = np.subtract(data,rf)

actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
date,,,,,,,,,,,
196307,-0.740000,-0.490000,-1.680000,2.020000,-0.960000,-0.500000,-1.300000,0.290000,0.530000,-1.880000,-0.390000
196308,4.610000,6.290000,5.940000,3.670000,4.880000,4.030000,6.140000,9.300000,3.940000,5.230000,5.060000
196309,-1.943333,-0.503333,-1.023333,-3.903333,-0.133333,2.096667,0.696667,-4.323333,-2.763333,-3.423333,-1.563333
196310,2.390000,9.450000,2.310000,-0.590000,8.020000,3.130000,0.210000,3.110000,-0.940000,1.110000,2.550000
196311,-1.406667,-5.116667,0.023333,-1.426667,-0.566667,3.883333,-1.506667,-1.926667,-1.296667,-0.046667,-0.856667


In [4]:
port_returns = []
port_variance = []
sharpe_ratio = []
stock_weights = []

In [5]:
# simulerer 1/N portfølje fra 1963-07 til 2004-11 som rollover med 60 måneders interval:
for month in range(0, len(returns_data) - 120):
    
    # Danner nyt data interval, returns og cov-matrix for hvert interval
    new_returns_data = returns_data[month:month + 120]
    returns_monthly = new_returns_data
    returns_mean = returns_monthly.mean()
    cov_matrix = returns_monthly.cov()
      
    # udregner profit, volatilitet og sharpe
    tl = np.dot(np.linalg.inv(cov_matrix), returns_mean)
    nl = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), returns_mean))
    weights = tl / nl
    
    # udregner afkast for måned T + 1 med de udregnede optimale vægte
    returns = np.dot(weights, returns_data[month + 120:month + 121].mean())
    variance = np.dot(weights, np.dot(cov_matrix, weights))
    sharpe = returns / np.sqrt(variance)

    # indsætter overstående udregninger i vektorerne
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_variance.append(variance)
    stock_weights.append(weights)

In [6]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Variance': port_variance,
             'Sharpe Ratio': sharpe_ratio}

In [7]:
# definerer de enkelte vægte af de valgte aktiver
for i in range(len((selected))):
    symbol = selected[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [11]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Variance', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in selected]
df = df[column_order]

df.head()

,Returns,Variance,Sharpe Ratio,NoDur Weight,Durbl Weight,Manuf Weight,Enrgy Weight,HiTec Weight,Telcm Weight,Shops Weight,Hlth Weight,Utils Weight,Other Weight,Mkt Weight
0,-13.889723,88.314092,-1.478014,3.985222,3.760662,11.320515,7.191923,4.444914,3.101023,4.339409,4.742751,2.919129,3.402605,-48.208152
1,-17.737827,101.471751,-1.760872,4.012448,3.765924,11.740059,7.280749,4.329822,3.152665,4.789998,5.005917,2.669640,3.634478,-49.381700
2,4.940111,142.437302,0.413928,4.688476,4.412031,14.948010,8.809922,5.249534,3.621592,5.823007,5.668858,3.326524,4.504710,-60.052664
3,2.886082,116.364130,0.267546,4.396391,4.077348,13.399570,8.139072,4.614815,3.222198,5.204115,5.216509,3.125303,4.296350,-54.691671
4,0.943056,155.478276,0.075631,4.281979,3.903277,15.479354,9.282262,4.903105,3.225811,6.162934,5.785829,3.477003,4.803835,-60.305390


In [9]:
df['Returns'].mean() / df['Returns'].std()

0.06996286380775166

In [10]:
df['Sharpe Ratio'].mean()

0.032665454264563604